In [2]:
import math
import pandas as pd
import numpy as np
import torch
from PIL import Image
from torch.utils.data import DataLoader
import torch.nn as nn

In [3]:
test_id = '1'
tb_dir = "runs/deap_swin_dep"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
channels = 32
samples = 384
eeg_data_folder = './DEAP/EEGData/'
test_eeg_data = np.load(f"{eeg_data_folder}s{str(test_id).zfill(2)}_eeg.npy")
test_labels = np.load(f"{eeg_data_folder}s{str(test_id).zfill(2)}_labels.npy")
available_subjects = [1, 2, 6, 7, 8, 9, 10, 12, 13, 16, 17, 18, 19, 20, 21, 22]
train_ids = [subject for subject in available_subjects if str(subject) != test_id]

[2, 6, 7, 8, 9, 10, 12, 13, 16, 17, 18, 19, 20, 21, 22]


In [13]:
'''
    -----------------------------合并训练集的所有eeg和label--------------------------------
'''
train_eeg_data = []
train_labels = []
for i in train_ids:
    eeg_file = f"{eeg_data_folder}s{str(i).zfill(2)}_eeg.npy"
    label_file = f"{eeg_data_folder}s{str(i).zfill(2)}_labels.npy"
    # 尝试加载 EEG 数据和标签，如果文件不存在则跳过
    try:
        eeg_data = np.load(eeg_file)
        labels = np.load(label_file)
        
        train_eeg_data.append(eeg_data)
        train_labels.append(labels)
    except FileNotFoundError:
        print(f"Data for subject {i} not found!!!!!!!!!!!!!!!!!")

train_eeg_data = np.concatenate(train_eeg_data, axis=0)
train_labels = np.concatenate(train_labels, axis=0)
'''
    -----------------------------组织图像数据,与eeg对齐--------------------------------
'''
# 创建一个列表，用于存储所有图像文件的路径
train_image_file_list = []
for i in train_ids:
    person_image_data_folder = f"./DEAP/faces/s{str(i).zfill(2)}/" 
    # 遍历每个人的图像文件
    for j in range(1, 801):
        filename = f"{person_image_data_folder}{j}.jpg"
        train_image_file_list.append(filename)

# eeg、图片组合在一起
train_combined_data = list(zip(train_eeg_data, train_image_file_list))


test_image_file_list = []
person_image_data_folder = f"./DEAP/faces/s{str(test_id).zfill(2)}/" 
for j in range(1, 801):
    filename = f"{person_image_data_folder}{j}.jpg"
    test_image_file_list.append(filename)
test_combined_data = list(zip(test_eeg_data, test_image_file_list))


data = train_combined_data

./DEAP/faces/s01/
800 800
./DEAP/faces/s01/1.jpg (32, 384)
12000


In [18]:
train_data = train_combined_data
train_labels = train_labels
test_data = test_combined_data
test_labels = test_labels

12000 32 ./DEAP/faces/s02/1.jpg
12000
800
800
